In [134]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KernelDensity
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [135]:
oil=pd.read_csv('olive.csv')
print(len(oil))
oil.head()

572


,id,Region,Area,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic,other
0,1,North-Apulia,1,1,1075,75,226,7823,672,36,60,29
1,2,North-Apulia,1,1,1088,73,224,7709,781,31,61,29
2,3,North-Apulia,1,1,911,54,246,8113,549,31,63,29
3,4,North-Apulia,1,1,966,57,240,7952,619,50,78,35
4,5,North-Apulia,1,1,1051,67,259,7771,672,50,80,46


In [136]:
oil.loc[oil['Region'] == 'North-Apulia', 'Region'] = 'extra virgin olive oil'
oil.loc[oil['Region'] == 'South-Apulia', 'Region'] = 'extra virgin olive oil'
oil.loc[oil['Region'] == 'Calabria', 'Region'] = 'organic extra virgin olive oil'
oil.loc[oil['Region'] == 'Sicily', 'Region'] = 'organic extra virgin olive oil'
oil.loc[oil['Region'] == 'Inland-Sardinia', 'Region'] = 'extra virgin olive oil'
oil.loc[oil['Region'] == 'Coast-Sardinia', 'Region'] = 'extra virgin olive oil'
oil.loc[oil['Region'] == 'Umbria', 'Region'] = 'extra virgin olive oil'
oil.loc[oil['Region'] == 'East-Liguria', 'Region'] = 'extra virgin olive oil'
oil.loc[oil['Region'] == 'West-Liguria', 'Region'] = 'extra virgin olive oil'
print(oil['Region'].unique())

['extra virgin olive oil' 'organic extra virgin olive oil']


In [137]:
oil.head()

,id,Region,Area,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic,other
0,1,extra virgin olive oil,1,1,1075,75,226,7823,672,36,60,29
1,2,extra virgin olive oil,1,1,1088,73,224,7709,781,31,61,29
2,3,extra virgin olive oil,1,1,911,54,246,8113,549,31,63,29
3,4,extra virgin olive oil,1,1,966,57,240,7952,619,50,78,35
4,5,extra virgin olive oil,1,1,1051,67,259,7771,672,50,80,46


In [138]:
oil=oil.drop('id' , axis= 1)
oil=oil.drop('other' , axis= 1)
oil=oil.drop('Area' , axis= 1)
oil.head(2)

,Region,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic
0,extra virgin olive oil,1,1075,75,226,7823,672,36,60
1,extra virgin olive oil,1,1088,73,224,7709,781,31,61


In [139]:
oil_organic = oil[oil['Region'] == 'organic extra virgin olive oil']
oil_extra = oil[oil['Region'] == 'extra virgin olive oil']

oil_organic.loc[oil_organic['Region'] == 'organic extra virgin olive oil', 'Region'] = 1
len(oil_organic)

92

In [ ]:
column_names = oil_organic.columns.tolist()


kde = KernelDensity(kernel='gaussian', bandwidth=0.5)
kde.fit(oil_organic)


n_samples = int(len(oil_organic) / 2)
synthetic_scaled = kde.sample(n_samples=n_samples, random_state=42)


generated_data = pd.DataFrame(synthetic_scaled, columns=column_names)


oil_organic = pd.concat([oil_organic, generated_data], ignore_index=True)


oil_organic['Region'] = 'organic extra virgin olive oil'
print(len(oil_organic))
oil_organic.head(100)

138


,Region,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic
0,organic extra virgin olive oil,2.000000,1315.000000,139.000000,230.000000,7299.000000,832.000000,42.000000,60.000000
1,organic extra virgin olive oil,2.000000,1321.000000,136.000000,217.000000,7174.000000,950.000000,43.000000,63.000000
2,organic extra virgin olive oil,2.000000,1359.000000,115.000000,246.000000,7234.000000,874.000000,45.000000,63.000000
3,organic extra virgin olive oil,2.000000,1378.000000,111.000000,272.000000,7127.000000,940.000000,46.000000,64.000000
4,organic extra virgin olive oil,2.000000,1295.000000,109.000000,245.000000,7253.000000,903.000000,43.000000,62.000000
...,...,...,...,...,...,...,...,...,...
95,organic extra virgin olive oil,2.406263,1255.678120,102.963995,223.501766,7395.180818,847.677440,47.180698,56.769018
96,organic extra virgin olive oil,2.782322,1354.690127,106.410951,236.043524,7208.850496,866.045880,47.006216,74.890164
97,organic extra virgin olive oil,2.738947,1355.740865,105.595753,235.749121,7209.457701,866.164376,47.735120,75.256634
98,organic extra virgin olive oil,2.484322,1274.648973,120.836169,214.803946,7284.268243,892.148060,40.130528,68.002557


In [141]:
oil = pd.concat([oil_organic, oil_extra])
oil = oil.reset_index(drop=True)
print(len(oil))
oil.head(200)

618


,Region,palmitic,palmitoleic,stearic,oleic,linoleic,linolenic,arachidic,eicosenoic
0,organic extra virgin olive oil,2.0,1315.0,139.0,230.0,7299.0,832.0,42.0,60.0
1,organic extra virgin olive oil,2.0,1321.0,136.0,217.0,7174.0,950.0,43.0,63.0
2,organic extra virgin olive oil,2.0,1359.0,115.0,246.0,7234.0,874.0,45.0,63.0
3,organic extra virgin olive oil,2.0,1378.0,111.0,272.0,7127.0,940.0,46.0,64.0
4,organic extra virgin olive oil,2.0,1295.0,109.0,245.0,7253.0,903.0,43.0,62.0
...,...,...,...,...,...,...,...,...,...
195,extra virgin olive oil,3.0,1449.0,175.0,198.0,6883.0,1162.0,40.0,70.0
196,extra virgin olive oil,3.0,1310.0,180.0,183.0,7054.0,1202.0,26.0,32.0
197,extra virgin olive oil,3.0,1360.0,163.0,176.0,6901.0,1280.0,28.0,65.0
198,extra virgin olive oil,3.0,1300.0,187.0,196.0,6920.0,1253.0,41.0,76.0


In [ ]:

X = oil.drop('Region', axis=1)
y = oil['Region']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model_precision = pd.DataFrame(columns=['Algorithm', 'precision'])


In [ ]:

gnb = GaussianNB()


gnb.fit(X_train, y_train)


y_pred = gnb.predict(X_test)

pred_df_features = pd.DataFrame(X_test)

pred_df = X_test.copy()
pred_df['Region'] = y_test
pred_df['Region_pred'] = y_pred

print(pred_df.head())


accuracy = accuracy_score(y_test, y_pred)
print("Naive bayes Accuracy:", accuracy)


new_row = pd.DataFrame([{'Algorithm': 'Naive bayes', 'precision': accuracy}])

print(accuracy)


model_precision = pd.concat([model_precision, new_row], ignore_index=True)

     palmitic  palmitoleic     stearic       oleic    linoleic    linolenic  \
49   2.000000  1296.000000  136.000000  260.000000  7380.00000   780.000000   
582  8.000000   980.000000   90.000000  330.000000  7540.00000  1040.000000   
82   4.000000  1153.000000   74.000000  316.000000  7593.00000   705.000000   
305  3.000000  1493.000000  204.000000  188.000000  6913.00000  1044.000000   
109  1.776743  1283.428199  102.107047  262.377131  7400.08659   763.192659   

     arachidic  eicosenoic                          Region  \
49   48.000000   51.000000  organic extra virgin olive oil   
582   0.000000    0.000000          extra virgin olive oil   
82   42.000000   64.000000  organic extra virgin olive oil   
305  32.000000   61.000000          extra virgin olive oil   
109  53.558071   65.076863  organic extra virgin olive oil   

                        Region_pred  
49   organic extra virgin olive oil  
582          extra virgin olive oil  
82   organic extra virgin olive oil  


<ipython-input-143-06ac9cb3fa2c>:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_precision = pd.concat([model_precision, new_row], ignore_index=True)


In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=3)


knn_model.fit(X_train, y_train)


y_pred = knn_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
new_row= pd.DataFrame([{'Algorithm': 'KNN', 'precision': accuracy}])
model_precision = pd.concat([model_precision, new_row], ignore_index=True)



Accuracy: 0.9483870967741935


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


knn_model = KNeighborsClassifier(n_neighbors=3)


knn_model.fit(X_train_scaled, y_train)


y_pred = knn_model.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


new_row = pd.DataFrame([{'Algorithm': 'KNN (z-score)', 'precision': accuracy}])
model_precision = pd.concat([model_precision, new_row], ignore_index=True)

Accuracy: 0.9612903225806452


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


lda = LinearDiscriminantAnalysis()
lda.fit(X_train_scaled, y_train)


y_pred = lda.predict(X_test_scaled)

accuracy = lda.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)


new_row = pd.DataFrame([{'Algorithm': 'LDA (z-score)', 'precision': accuracy}])
model_precision = pd.concat([model_precision, new_row], ignore_index=True)

Accuracy: 0.9290322580645162


In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)


y_pred = lda.predict(X_test)


accuracy = lda.score(X_test, y_test)

print("Accuracy:", accuracy)
new_row = pd.DataFrame([{'Algorithm': 'LDA', 'precision': accuracy}])
model_precision = pd.concat([model_precision, new_row], ignore_index=True)



Accuracy: 0.9290322580645162


In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
new_row= pd.DataFrame([{'Algorithm': 'Decision Tree', 'precision': accuracy}])
model_precision = pd.concat([model_precision, new_row], ignore_index=True)



Accuracy: 0.9483870967741935


In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(70,), max_iter=1000 , activation= 'logistic') # we try different values for the number of layers 10-> 0.24 5 -> 0.75 100 -> 0.8 70 -> 0.91


clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)
new_row= pd.DataFrame([{'Algorithm': 'ANN', 'precision': accuracy}])
model_precision = pd.concat([model_precision, new_row], ignore_index=True)



Accuracy: 0.9290322580645162


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp = MLPClassifier(hidden_layer_sizes=(80,20 ), max_iter=300, random_state=42)
mlp.fit(X_train_scaled, y_train)


y_pred = mlp.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9612903225806452


In [157]:
print(model_precision)

       Algorithm  precision
0    Naive bayes   0.896774
1            KNN   0.948387
2  KNN (z-score)   0.961290
3  LDA (z-score)   0.929032
4            LDA   0.929032
5  Decision Tree   0.948387
6            ANN   0.929032
